In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.path as mpath
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
#from xgcm import Grid
import requests
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

import dask.array as da
from dask.diagnostics import ProgressBar

In [2]:
col_height = xr.open_dataset(f'Data_Step2/control/col_height.nc').rename({'yh':'lat', 'xh':'lon'})['col_height']

# Ensure that longitude coords are from -180 to 180
col_height.coords['lon'] = (col_height.coords['lon'] + 180) % 360 - 180
col_height = col_height.sortby(col_height.lon)

In [3]:
experiment = 'historical'
var = 'uo'

In [4]:
if experiment == 'control':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_piControl_c192_OM4p25_v8/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc')
elif experiment == 'historical':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_historical_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc')
elif experiment == 'ssp585':
    path='/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ts/monthly/5yr/'
    ds_grid=xr.open_dataset('/archive/Raphael.Dussin/FMS2019.01.03_devgfdl_20221223/CM4_ssp585_c192_OM4p25/gfdl.ncrc4-intel18-prod-openmp/pp/ocean_monthly_z/ocean_monthly_z.static.nc')

dsShortGrid=ds_grid.isel(xq=np.arange(1,1441)).isel(yq=np.arange(1,1081))

In [5]:
test=glob.glob(path + f'ocean_monthly_z.*.{var}.nc')
test[0][163:176]

'185501-185912'

In [6]:
test=glob.glob(path + f'ocean_monthly_z.*.{var}.nc')

if experiment == 'control':
    year_name_start_idx, year_name_end_idx = 165, 178
elif experiment == 'historical':
    year_name_start_idx, year_name_end_idx = 163, 176
elif experiment == 'ssp585':
    year_name_start_idx, year_name_end_idx = 159, 172

one=test[0]
times=[one[year_name_start_idx:year_name_end_idx]]

for tt in np.arange(1,len(test)): 
    one=test[tt]
    times=times+[one[year_name_start_idx:year_name_end_idx]]
    
times = sorted(times)

if experiment == 'control':
    pass
elif experiment == 'historical':
    times = times[-5:]
elif experiment == 'ssp585':
    times = times[-4:]
    #times = times[-12:-8]

times

['199001-199412',
 '199501-199912',
 '200001-200412',
 '200501-200912',
 '201001-201412']

In [7]:
datasets = []

if experiment == 'control':
    pass
elif experiment == 'historical':
    start_yr, end_yr = '1992', '2013'
elif experiment == 'ssp585':
    pass

files = [f'{path}/ocean_monthly_z.{time}.{var}.nc' for time in times]

print('Gathering dataset...')
ds = xr.open_mfdataset(files, chunks={"time": 12}, parallel=True)
with ProgressBar():
    ds = ds.compute()
    
datasets.append(ds)
    
#datasets.append(ds_grid)

print('Merging datasets...')
if experiment == 'control':
    full_ds = xr.merge(datasets).compute()
elif experiment == 'historical':
    full_ds = xr.merge(datasets).sel(time=slice(start_yr, end_yr)).compute()
elif experiment == 'ssp585':
    full_ds = xr.merge(datasets).compute()

full_ds

Gathering dataset...
[########################################] | 100% Completed | 275.97 s
Merging datasets...


<xarray.Dataset>
Dimensions:     (time: 264, nv: 2, z_l: 35, yh: 1080, xq: 1441, z_i: 36)
Coordinates:
  * nv          (nv) float64 1.0 2.0
  * time        (time) object 1992-01-16 12:00:00 ... 2013-12-16 12:00:00
  * xq          (xq) float64 -299.8 -299.6 -299.3 -299.1 ... 59.66 59.91 60.16
  * yh          (yh) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * z_i         (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
  * z_l         (z_l) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
Data variables:
    average_DT  (time) timedelta64[ns] 31 days 28 days ... 30 days 31 days
    average_T1  (time) datetime64[ns] 1991-11-28 1991-12-29 ... 2013-10-22
    average_T2  (time) datetime64[ns] 1991-12-29 1992-01-26 ... 2013-11-22
    time_bnds   (time, nv) object 1992-01-01 00:00:00 ... 2014-01-01 00:00:00
    uo          (time, z_l, yh, xq) float32 nan nan nan nan ... nan nan nan nan
Attributes:
    filename:          ocean_monthly_z.199001-199412.uo.nc
    title:             CM4_historical_c192_OM4p25
    associated_files:  areacello: 19900101.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A

In [8]:
full_ds = full_ds.mean('time')
full_ds[var]

<xarray.DataArray 'uo' (z_l: 35, yh: 1080, xq: 1441)>
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * xq       (xq) float64 -299.8 -299.6 -299.3 -299.1 ... 59.66 59.91 60.16
  * yh       (yh) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * z_l      (z_l) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03

In [9]:
full_ds[var].to_netcdf(f'Data_Step2/{experiment}/{var}.nc')